# Payment Channel - BX exercise

In this BX tutorial, we will demonstrate a simple payment channel between two participants, Alice and Bob.

* Wallet Setup for Alice & Bob
* 1) Opening a payment channel
* 2) Updating the payment channel
* 3) Alice broadcasts old state. Bob sweeps the channel.



<hr style="border: 0.5px dashed #000;">


## Wallet setup for Alice & Bob

In [1]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)

Alice keys: `m/44'/1'/1'/0/5`

In [7]:
# hd_m_44h_1h_1h_0_5=

# privatekey_44h_1h_1h_0_5=
# publickey_44h_1h_1h_0_5=
# publickeyhash_44h_1h_1h_0_5=
# address_44h_1h_1h_0_5=

hd_m_44h_1h_1h_0_5=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 5)

privatekey_44h_1h_1h_0_5=$(bx hd-to-ec $hd_m_44h_1h_1h_0_5)
publickey_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_5=$(bx sha256 $publickey_44h_1h_1h_0_5 | bx ripemd160)
address_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_5
echo $publickey_44h_1h_1h_0_5
echo $publickeyhash_44h_1h_1h_0_5


mqpVyrj3iRPe7sNKht9vvnbhnDCAPdiJYv
025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae84
7102c7914f7ac4e5a8055c5954b3462e0ddb419f


Bob keys: `m/44'/1'/1'/0/6`

In [8]:
# hd_m_44h_1h_1h_0_6=

# privatekey_44h_1h_1h_0_6=
# publickey_44h_1h_1h_0_6=
# publickeyhash_44h_1h_1h_0_6=
# address_44h_1h_1h_0_6=

hd_m_44h_1h_1h_0_6=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 6)

privatekey_44h_1h_1h_0_6=$(bx hd-to-ec $hd_m_44h_1h_1h_0_6)
publickey_44h_1h_1h_0_6=$(bx hd-to-public $hd_m_44h_1h_1h_0_6 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_6=$(bx sha256 $publickey_44h_1h_1h_0_6 | bx ripemd160)
address_44h_1h_1h_0_6=$(bx hd-to-public $hd_m_44h_1h_1h_0_6 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_6
echo $publickey_44h_1h_1h_0_6
echo $publickeyhash_44h_1h_1h_0_6

mwLPgbtKttz63WkW5RvKpzjng64Kdzo3z7
036652acb7231efa990a5220c209d2bf05351fba53d0eb609fcaba4dee06983126
ad82a1640cf5b64aaa877e74e9824181882e5130


### Fetch previous UTXO's Alice is spending.

In [4]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"

bx fetch-tx 11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea -f json | jq ".transaction.outputs[5]"

{
  "address_hash": "7102c7914f7ac4e5a8055c5954b3462e0ddb419f",
  "script": "dup hash160 [7102c7914f7ac4e5a8055c5954b3462e0ddb419f] equalverify checksig",
  "value": "499550"
}


In [5]:
# previous_txid=
# previous_output_index=
# previous_output_amount=

previous_txid=11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea
previous_output_index=5
previous_output_amount=499550


<hr style="border: 0.5px dashed #000;">

## 1) Opening the payment channel
<br>
<img src="images/payment_channel_funding.jpg" alt="drawing" style="" width="700px"/>

## Alice creates the funding transaction

### A + B multisig output script
* `2`
* `public key A`
* `public key B`
* `2`
* `checkmultisig`


In [9]:
# bx script-encode "2 [pubkey_alice] [pubkey_bob] 2 checkmultisig"

# funding_output_script=

funding_output_script=$(bx script-encode "2 [$publickey_44h_1h_1h_0_5] [$publickey_44h_1h_1h_0_6] 2 CHECKMULTISIG")
echo $funding_output_script


5221025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae8421036652acb7231efa990a5220c209d2bf05351fba53d0eb609fcaba4dee0698312652ae


### Calculate output amount

* `100 Satoshi/sigops + 1 Satoshi/Byte` 
* Non-segwit multisig counts as `4 * 20 sigops`


In [10]:
# tx bytes: 4 + 1 + 1*(32+4+1+(1+72+1+33)+4) + 1 + 1*(8+1+(1+34+34+1+1)) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 148 + 1 + 1 \* 80 + 4)
echo $tx_byte_count


238


In [11]:
# Output amount total.

# funding_output_amount=

funding_output_amount=$(expr $previous_output_amount - 20 \* 400 - $tx_byte_count \* 1)
echo $funding_output_amount


491312


### Encode transaction for signing


In [12]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# funding_tx=

funding_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $funding_output_script:$funding_output_amount)
echo $funding_tx


0100000001ea03ec3bcb3417c58ab4d61677d26498f974fd979734e3697ffc77507f58a8110500000000ffffffff01307f070000000000475221025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae8421036652acb7231efa990a5220c209d2bf05351fba53d0eb609fcaba4dee0698312652ae00000000


### Alice signs funding transaction.

**Fetch previous output script**

In [13]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[5].script")
echo $previous_output_script


dup hash160 [7102c7914f7ac4e5a8055c5954b3462e0ddb419f] equalverify checksig


**Sign transaction with previous output script.**

In [14]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 

signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_5 "$previous_output_script" $funding_tx)


**Set the input script into the finalised transaction.**

In [28]:
# bx input-set "[signature] [public key point]" [transaction template]

# funding_tx=

funding_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_1h_0_5]" $funding_tx)


In [29]:
# Validate.

bx validate-tx $funding_tx

The transaction is valid.


In [75]:
#funding_txid=

funding_txid=$(bx bitcoin256 $funding_tx)
echo $funding_txid


625c3161f282a2c90d0710e1100cce3743beaeaf93c6dde39edcce13bc82daa1


**Important:** Alice can only broadcast the funding transaction once commitment 0 has been signed by both parties, as in the following section. Otherwise, her funds could become unspendable without Bob's cooperation. 

 ## 2) Commitment transactions `0a/0b`


Alice transfers 50000 satoshi to Bob.

### Alice's Commitment transaction `0a`



#### Output Scripts
* RSMC A
* PKPKH B

#### RSMC A
* `IF`
    * `[public key R]`
* `ELSE`
    * `[3-byte delay]`
    * `checksequenceverify`
    * `drop`
    * `[public key A]`
* `ENDIF`
* `CHECKSIG`

#### `Secret r = r_A + r_B`
* Both Alice and Bob independently generate secrets `r_A` and `r_B`
* Alice and Bob only exchange public keys `R_A` AND `R_B`
* `public key R = R_A + R_B = r_A * G + r_B * G`

#### Alice computes `public key R` by adding `R_B` to` r_A * G:`


In [34]:
# publickey_R=

publickey_R=$(bx ec-add $publickey_44h_1h_1h_0_6 $privatekey_44h_1h_1h_0_5)
echo $publickey_R


038cd1701e7c0140f911db0d7d0c21286977bf8cb4f9c6f7eb2141bb70c68addcc


#### Encoding Output Scripts for Commitment `0a`

In [36]:
# RSMC_A_output_script=

RSMC_A_output_script=$(bx script-encode "IF [$publickey_R] ELSE [050000] CHECKSEQUENCEVERIFY DROP [$publickey_44h_1h_1h_0_5] ENDIF CHECKSIG")


In [37]:
# P2PKH_B_output_script=

P2PKH_B_output_script=$(bx script-encode "DUP HASH160 [$publickeyhash_44h_1h_1h_0_6] EQUALVERIFY CHECKSIG")


#### Calculate output amount.

In [66]:
# tx bytes: 4 + 1 + 1*(32+4+1+(1+73+73)+4) + 1 + 1*(8+1+(1+34+1+4+1+1+34+1+1)) + 1*(8+1+25) + 4 

# (32+4+1+1+72+1+33+4)

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 188 + 1 + 1 \* 87 + 1 \* 34 + 4)
echo $tx_byte_count


319


In [91]:
total_output_amount=$(expr $funding_output_amount - 2 \* 400 - $tx_byte_count \* 1)
echo $total_output_amount

#RSMC_A_output_amount=
RSMA_A_output_amount=$(expr $total_output_amount - 50000)

#P2PKH_B_output_amount=
P2PKH_B_output_amount=50000


490193


**Encode Commitment `0a` for signing**

In [92]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# tx_0a=

tx_0a=$(bx tx-encode \
--input $funding_txid:0:4294967295 \
--output $RSMC_A_output_script:$RSMA_A_output_amount \
--output $P2PKH_B_output_script:$P2PKH_B_output_amount)

echo $tx_0a

0100000001a1da82bc13cedc9ee3ddc693afaebe4337ce0c10e110070dc9a282f261315c620000000000ffffffff0281b70600000000004e6321038cd1701e7c0140f911db0d7d0c21286977bf8cb4f9c6f7eb2141bb70c68addcc6703050000b27521025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae8468ac50c30000000000001976a914ad82a1640cf5b64aaa877e74e9824181882e513088ac00000000


**Sign commitment transaction `0a` by Alice & Bob**

In [93]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $funding_txid \
| jq -r ".transaction.outputs[0].script")
echo $previous_output_script


2 [025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae84] [036652acb7231efa990a5220c209d2bf05351fba53d0eb609fcaba4dee06983126] 2 checkmultisig


In [94]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature_a= 
# signature_b= 

signature_a=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_5 "$previous_output_script" $tx_0a)
signature_b=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_6 "$previous_output_script" $tx_0a)


In [95]:
# bx input-set "[signature] [public key point]" [transaction template]

# tx_0a=

tx_0a=$(bx input-set --index 0 "zero [$signature_a] [$signature_b]" $tx_0a)
echo $tx_0a


0100000001a1da82bc13cedc9ee3ddc693afaebe4337ce0c10e110070dc9a282f261315c6200000000920047304402206894240bf28079c36706741f0b6b5db2d56b53ae146b9fc5ab07793817e755c70220461c638e37b33afa1beade3ccaf16e94d46ae284a7d822936cc7e07cb9efd3f10148304502210089159531f51b19197d640682059f4d7388ffe0d71434b4f662e1c37a358aefa0022067c509063747b6fe27586c511908db725559fb8117d4aeb749486f4e319d595501ffffffff0281b70600000000004e6321038cd1701e7c0140f911db0d7d0c21286977bf8cb4f9c6f7eb2141bb70c68addcc6703050000b27521025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae8468ac50c30000000000001976a914ad82a1640cf5b64aaa877e74e9824181882e513088ac00000000


In [108]:
# tx_0a_txid=

tx_0a_txid=$(bx bitcoin256 $tx_0a)


In [120]:
# Validate.

bx validate-tx $tx_0a


The transaction is valid.


### Bob's commitment transaction `0b`
* Omitted here for brevity's sake: Left to student to complete.
* Follows same steps as Alice's commitment transaction `0a`

<hr style="border: 0.5px dashed #000;">


## 2) Updating the channel
<br>
<img src="images/payment_channel_update.jpg" alt="drawing" style="" width="700px"/>

Alice spends another 50000 to bob.

### A. New commitment transaction `1a`/`1b` are created
* Not shown 
* Follows previous section.


### B. Alice reveals `r_A` to Bob
* Since Alice is paying Bob, she must revoke her past commitment TX output. 
* Bob can now compute `r` = `r_A + r_B`

In [99]:
# secret_r=

secret_r=$(bx ec-add-secrets $privatekey_44h_1h_1h_0_5 $privatekey_44h_1h_1h_0_6)
echo $secret_r
echo $publickey_R


a5132279616a36f8d3b2849a99fd44506836e3116fa8c650b6fadd3b9e2d8840
038cd1701e7c0140f911db0d7d0c21286977bf8cb4f9c6f7eb2141bb70c68addcc


In [100]:
# Verify computed secret r corresponds to public key R.

bx ec-to-public $secret_r


038cd1701e7c0140f911db0d7d0c21286977bf8cb4f9c6f7eb2141bb70c68addcc


<hr style="border: 0.5px dashed #000;">

## 3) Alice broadcasts old state. Bob sweeps the channel.
<br>
<img src="images/payment_channel_penalty.jpg" alt="drawing" style="" width="700px"/>


### A. Alice broadcasts commitment tx `0a`

In [121]:
echo $tx_0a
bx send-tx $tx_0a


0100000001a1da82bc13cedc9ee3ddc693afaebe4337ce0c10e110070dc9a282f261315c6200000000920047304402206894240bf28079c36706741f0b6b5db2d56b53ae146b9fc5ab07793817e755c70220461c638e37b33afa1beade3ccaf16e94d46ae284a7d822936cc7e07cb9efd3f10148304502210089159531f51b19197d640682059f4d7388ffe0d71434b4f662e1c37a358aefa0022067c509063747b6fe27586c511908db725559fb8117d4aeb749486f4e319d595501ffffffff0281b70600000000004e6321038cd1701e7c0140f911db0d7d0c21286977bf8cb4f9c6f7eb2141bb70c68addcc6703050000b27521025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae8468ac50c30000000000001976a914ad82a1640cf5b64aaa877e74e9824181882e513088ac00000000
Sent transaction.


### B. Bob sweeps funds from commitment tx `0a`

#### Encode P2PKH output script

In [123]:
# Encode output spendable by Bob

# penalty_output_script=

penalty_output_script=$(bx script-encode "DUP HASH160 [$publickeyhash_44h_1h_1h_0_6] EQUALVERIFY CHECKSIG")


#### Calculate output amount

In [124]:
# tx bytes: 4 + 1 + 1*(32+4+1+(1+73)+4) + 1*(32+4+1+(73+34)+4) + 1 + 1*(8+1+25) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 115 + 1 \* 148 + 1 + 1 \* 34 + 4)
echo $tx_byte_count


307


In [125]:
# Output amount total of both inputs being swept.

# penalty_output_amount=

penalty_output_amount=$(expr $RSMA_A_output_amount + $P2PKH_B_output_amount - 1 \* 400 - $tx_byte_count \* 1)
echo $penalty_output_amount


489486


#### Encode penalty transaction for signing.

In [126]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# penalty_tx=

penalty_tx=$(bx tx-encode \
--input $tx_0a_txid:0:4294967295 \
--input $tx_0a_txid:1:4294967295 \
--output $penalty_output_script:$penalty_output_amount)
echo $penalty_tx


0100000002895edbe83ea4013c25a44d05ae6bc5a440d5b27093761bd61b763bf7a219eda10000000000ffffffff895edbe83ea4013c25a44d05ae6bc5a440d5b27093761bd61b763bf7a219eda10100000000ffffffff010e780700000000001976a914ad82a1640cf5b64aaa877e74e9824181882e513088ac00000000


#### Sign the transaction.


* Fetch previous `RSMC_A output script` from `tx_0a` input `0`

In [136]:
# previous_RSMA_A_script=$(bx fetch-tx ... )

previous_RSMA_A_script=$(bx fetch-tx -f json $tx_0a_txid \
| jq -r ".transaction.outputs[0].script")
echo $previous_RSMA_A_script


if [038cd1701e7c0140f911db0d7d0c21286977bf8cb4f9c6f7eb2141bb70c68addcc] else [050000] checksequenceverify drop [025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae84] endif checksig


* Fetch previous `P2PKH_B output script` from `tx_0a` input `1`

In [137]:
# previous_P2PKH_B_script=$(bx fetch-tx ... )

previous_P2PKH_B_script=$(bx fetch-tx -f json $tx_0a_txid \
| jq -r ".transaction.outputs[1].script")
echo $previous_P2PKH_B_script


dup hash160 [ad82a1640cf5b64aaa877e74e9824181882e5130] equalverify checksig


In [138]:
# signature_RSMC_A=

signature_RSMC_A=$(bx input-sign --sign_type all --index 0 $secret_r "$previous_RSMA_A_script" $penalty_tx)


In [139]:
# signature_RSMC_B=

signature_P2PKH_B=$(bx input-sign --sign_type all --index 1 $privatekey_44h_1h_1h_0_6 "$previous_P2PKH_B_script" $penalty_tx)


**Set the input script into the finalised transaction.**

Input script to spend RSMC A.
* `[Sig RSMC R]`
* `1`

Input script to spend P2PKH B is
* `[Sig P2PKH B]`
* `[public key B]`


In [140]:
# bx input-set "[signature] [public key point]" [transaction template]

# Set first input
# penalty_tx=

# Set second input
# penalty_tx=

penalty_tx=$(bx input-set --index 0 "[$signature_RSMC_A] 1" $penalty_tx)
penalty_tx=$(bx input-set --index 1 "[$signature_P2PKH_B] [$publickey_44h_1h_1h_0_6]" $penalty_tx)


In [142]:
# Validate penalty tx.
bx validate-tx $penalty_tx


The transaction is valid.
